In [ ]:
os.chdir('/content')

!git clone https://github.com/dgrifka/baseball_game_simulator.git /content/baseball_game_simulator

os.chdir('/content/baseball_game_simulator')

In [ ]:
import pandas as pd

from Simulator.get_game_information import team_info, fetch_games, get_game_info
from Simulator.game_simulator import outcomes, calculate_total_bases, simulator, create_detailed_outcomes_df, outcome_rankings
from Simulator.visualizations import run_dist, la_ev_graph, create_estimated_bases_table

import matplotlib.pyplot as plt
from Simulator.constants import team_colors, mlb_team_logos

from Simulator.utils import reload_module_from_file, reload_functions_from_module

import os

In [ ]:
folder_path = 'your folder path'

In [ ]:
## See teams below
teams_df = team_info()

## How far back you want to go for games
days_ago = 1

games_df, games_list = fetch_games(days_ago, all_columns = False)
## Remove games we've already created tweets for
# games_list = [game for game in games_list if game not in existing_game_pks]

## Add short team names
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.away.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'away.team'})
games_df = pd.merge(games_df, teams_df[['team.id', 'teamName']], left_on='teams.home.team.id', right_on='team.id')
games_df = games_df.rename(columns={'teamName': 'home.team'})
games_df = games_df.drop(columns=['team.id_x', 'team.id_y'])
games_df = games_df.dropna(subset=['teams.home.score']).reset_index(drop=True)

In [ ]:
## If you want to manually choose the games
# games_list = []

In [ ]:
game_info_list = []
for game_id in games_list:
  # Gather hit data, all data (all_bb), and non-batted balls
  result = get_game_info(game_id, all_columns=False)

  if result is None:
      print(f"Error retrieving data for game ID: {game_id}")
      continue

  game_data, all_bb, steals_and_pickoffs = result

  if game_data is None:
      print(f"Error retrieving data for game ID: {game_id}")
      continue
  ## venue equals value from game_data where gamePk == game_id
  venue = games_df[games_df['gamePk'] == game_id]['venue.name'].values[0]
  game_data['venue.name'] = venue
  ## Get actual game info
  game_info = games_df.copy()
  game_info = game_info[game_info['gamePk'] == game_id]
  if len(game_info) == 0:
      print(f"No game found for game ID: {game_id}")
      continue
  home_team = game_info['home.team'].values[0]
  away_team = game_info['away.team'].values[0]
  home_score = int(game_info['teams.home.score'].iloc[0])
  away_score = int(game_info['teams.away.score'].iloc[0])
  print(f"{away_team} @ {home_team}")

  home_outcomes = outcomes(game_data, steals_and_pickoffs, 'home')
  away_outcomes = outcomes(game_data, steals_and_pickoffs, 'away')

  # Calculate the estimated total bases
  home_estimated_total_bases = calculate_total_bases(home_outcomes)
  away_estimated_total_bases = calculate_total_bases(away_outcomes)

  # For home team
  home_detailed_df = create_detailed_outcomes_df(game_data, steals_and_pickoffs, 'home')
  home_detailed_df['team'] = home_team
  # For away team
  away_detailed_df = create_detailed_outcomes_df(game_data, steals_and_pickoffs, 'away')
  away_detailed_df['team'] = away_team

  ## Get dataframes for estimated bases graphs
  top_15_bases = outcome_rankings(home_detailed_df, away_detailed_df)

  ## Clean up the tuples
  home_outcomes = [outcome[0] if isinstance(outcome[0], tuple) else outcome[0] for outcome in home_outcomes]
  away_outcomes = [outcome[0] if isinstance(outcome[0], tuple) else outcome[0] for outcome in away_outcomes]

  # Get the last value from the officialDate column, since there could be multiple games
  ## We might need to add gamePk in the save, since multiple games can be played on the same day by the same teams
  game_date = pd.to_datetime(game_info['officialDate'].iloc[-1])
  formatted_date = game_date.strftime("%m/%d/%Y")

  num_simulations = 2000
  home_runs_scored, away_runs_scored, home_win_percentage, away_win_percentage, tie_percentage = simulator(num_simulations, home_outcomes, away_outcomes)
  la_ev_graph(home_outcomes, away_outcomes, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage, mlb_team_logos, formatted_date, folder_path)
  run_dist(num_simulations, home_runs_scored, away_runs_scored, home_team, away_team, home_score, away_score, home_win_percentage, away_win_percentage, tie_percentage, formatted_date, folder_path)
  create_estimated_bases_table(top_15_bases, away_team, home_team, away_score, home_score, away_win_percentage, home_win_percentage, formatted_date, folder_path)